In [2]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\src")
    sys.path.append(module_path+"\\References")


import data_preprocessing as dp
import metrics
import genetic_algorithm as ga

In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

In [4]:
data = pd.read_excel('/content/final_data.xlsx')
data.head()

,Unnamed: 0,Team size**1 x Actual duration**1,Dedicated team members**1 x Actual duration**1,Actual duration**1 x Development environment adequacy**1 x 11-20**1,Team size**1 x 21-30**1 x Inaccurate requirements specifications and required the re-analysis of the software requirements**1,User resistance**1 x Project manager**1 x 21-30**1,Development environment adequacy**1 x 21-30**1 x Inaccurate requirements specifications and required the re-analysis of the software requirements**1,Actual duration**1 x 21-30**1 x Inaccurate requirements specifications and required the re-analysis of the software requirements**1,Requirment stability**1 x 21-30**1 x Inaccurate requirements specifications and required the re-analysis of the software requirements**1,Dedicated team members**1 x Technical stability**1 x Tool availability**1,...,Dedicated team members**1 x Actual duration**1 x Economic instability impact**1,Technical stability**1 x 21-30**1 x Accurate requirements specifications used to develop the software system**1,Dedicated team members**1 x Actual duration**1 x Accurate requirements specifications used to develop the software system**1,Dedicated team members**1 x Users stability**1 x 1-5**1,Object points**1 x Project manager**1 x >500**1,Team size**1 x Developer**1 x >500**1,Dedicated team members**1 x Users stability**1 x Accurate requirements specifications used to develop the software system**1,Dedicated team members**1 x Team cohesion**1 x Developer incentives policy**1,Requirment stability**1 x Tool availability**1 x 11-20**1,Actual effort
0,0,18.0,18.0,0,0,0.0,0,0,0.0,42,...,18.0,0,18.0,0.0,0,6,11.333333,96,0.000000,3168.0
1,1,9.0,9.0,0,0,0.0,0,0,0.0,24,...,27.0,0,0.0,0.0,0,0,0.000000,108,0.000000,1584.0
2,2,30.0,30.0,0,0,0.0,0,0,0.0,48,...,60.0,0,0.0,0.0,0,0,0.000000,180,0.000000,5280.0
3,3,36.0,24.0,6,0,0.0,0,0,0.0,32,...,24.0,0,24.0,0.0,0,0,20.000000,16,10.031746,5280.0
4,4,216.0,216.0,24,0,0.0,0,0,0.0,36,...,648.0,0,216.0,0.0,0,0,17.000000,54,18.603175,38016.0


In [5]:
target = data['Actual effort']
X = data.drop(['Unnamed: 0', 'Actual effort'], axis = 1)

# Support Vector Regression

In [6]:
from sklearn.svm import LinearSVR

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 0)

In [8]:
X_train, X_test = pd.DataFrame(X_train, columns = X.columns), pd.DataFrame(X_test, columns = X.columns)

In [9]:
scaler, X_train = dp.scaling_numerical_data(X_train.columns, X_train)
X_test = scaler.transform(X_test)

In [10]:
y_scaler = StandardScaler()
y_train_sc = y_scaler.fit_transform(y_train.to_numpy().reshape((-1, 1)))
y_test_sc = y_scaler.transform(y_test.to_numpy().reshape((-1, 1)))

In [11]:
from sklearn.utils.validation import column_or_1d

In [12]:
y_train, y_test = column_or_1d(y_train), column_or_1d(y_test)

## Cross Validation score

#### 10k fold cross validation

In [ ]:
r2 = cross_val_score(LinearSVR(random_state = 0, verbose = 0), X_train, column_or_1d(y_train_sc), cv = 10).mean()
mae = cross_val_score(LinearSVR(random_state = 0, verbose = 0), X_train, column_or_1d(y_train_sc), cv = 10, scoring = 'neg_mean_absolute_error').mean()

In [23]:
print('10k fold cross validation r2 score: {}'.format(r2))
print('10k fold cross validation mean absolute error: {}'.format(np.abs(mae)))

10k fold cross validation r2 score: 0.9956045984486694
10k fold cross validation mean absolute error: 0.02492910342519905


#### 5k fold cross validation

In [ ]:
r2 = cross_val_score(LinearSVR(random_state = 0, verbose = 0), X_train, column_or_1d(y_train_sc), cv = 5).mean()
mae = cross_val_score(LinearSVR(random_state = 0, verbose = 0), X_train, column_or_1d(y_train_sc), cv = 5, scoring = 'neg_mean_absolute_error').mean()

In [25]:
print('10k fold cross validation r2 score: {}'.format(r2))
print('10k fold cross validation mean absolute error: {}'.format(np.abs(mae)))

10k fold cross validation r2 score: 0.994278015805372
10k fold cross validation mean absolute error: 0.03224845802016403


# Modeling


In [ ]:
svr_model = LinearSVR()
svr_model.fit(X_train, y_train_sc)
svr_model.score(X_train, y_train_sc), svr_model.score(X_test, y_test_sc)

#### Metrics

In [ ]:
train_predictions = y_scaler.inverse_transform(svr_model.predict(X_train).reshape((-1, 1))).reshape((len(y_train), ))
test_predictions = y_scaler.inverse_transform(svr_model.predict(X_test).reshape((-1, 1))).reshape((len(y_test), ))

In [ ]:
metrics.scores_DataFrame(y_train, y_test, train_predictions, test_predictions)

In [ ]:
metrics.plot_predictions(y_train, train_predictions, (20, 10), 'Train set')
metrics.plot_predictions(y_test, test_predictions, (20, 10), 'Test set')

# Optimize LinearSVR paramaters with GA-Algorithms


In [ ]:
params = {'epsilon': np.linspace(0.1, 1, 10), 'tol': [0.00001, 0.0001, 0.001, 0.01, 0.1], 'random_state': [0], 'max_iter': [10, 100, 1000, 10000], 'C': [1]}

In [ ]:
ga_alg = ga.GeneticAlgorithm(LinearSVR(), params, X_train, y_train_sc, 10, 0.9997631174256918, 5)

In [ ]:
populations = ga_alg.evolution()

In [ ]:
svr_model = LinearSVR(**populations[0])
svr_model.fit(X_train, y_train_sc)

In [ ]:
svr_model.score(X_train, y_train_sc), svr_model.score(X_test, y_test_sc)